##### Copyright 2020 The TensorFlow IO Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# TensorFlow を使用した Azure Blob Storage

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/io/tutorials/azure"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">TensorFlow.org で表示</a></td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/ja/io/tutorials/azure.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Google Colab で実行</a>   </td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/ja/io/tutorials/azure.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">GitHub でソースを表示</a>   </td>
      <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/ja/io/tutorials/azure.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">ノートブックをダウンロード</a>   </td>
</table>

注: このノートブックは、Python パッケージのほか、`npm install --user` を使用してパッケージをインストールします。ローカルで実行する際には注意してください。


## 概要

このチュートリアルでは、TensorFlow を使って [Azure Blob Storage](https://azure.microsoft.com/en-us/services/storage/blobs/) 上のファイルの読み取りと書き込みを使用する方法を説明します。TensorFlow IO の Azure ファイルシステム統合を使用します。

Azure Blob Storage のファイルの読み取りと書き込みには、Azure Storage アカウントが必要です。Azure Storage キーは、環境変数で指定します。

```
os.environ['TF_AZURE_STORAGE_KEY'] = '<key>'
```

Storage アカウント名とコンテナ名は、ファイル名 URL の一部です。

```
azfs://<storage-account-name>/<container-name>/<path>
```

このチュートリアルは実演を目的としているため、オプションとして Azure Storage のエミュレータである [Azurite](https://github.com/Azure/Azurite) をセットアップできます。Azurite エミュレータでは、TensorFlow を使用して、Azure Blob Storage を介したファイルの読み取りと書き込みを行えます。

## セットアップと使用方法

### 必要なパッケージをインストールし、ランタイムを再起動する

In [2]:
try:
  %tensorflow_version 2.x 
except Exception:
  pass

!pip install tensorflow-io

TensorFlow 2.x selected.
     |████████████████████████████████| 20.1MB 42.7MB/s 


### Azurite のインストールとセットアップ（オプション）

Azure Storage アカウントをお持ちでない場合に、Azure Storage インターフェースをエミュレートする Azurite をインストールしてセットアップするには次を行う必要があります。

In [3]:
!npm install azurite@2.7.0

npm WARN deprecated request@2.87.0: request has been deprecated, see https://github.com/request/request/issues/3142
npm WARN saveError ENOENT: no such file or directory, open '/content/package.json'
npm notice created a lockfile as package-lock.json. You should commit this file.
npm WARN enoent ENOENT: no such file or directory, open '/content/package.json'
npm WARN content No description
npm WARN content No repository field.
npm WARN content No README data
npm WARN content No license field.

+ azurite@2.7.0
added 116 packages from 141 contributors in 6.591s


In [4]:
# The path for npm might not be exposed in PATH env,
# you can find it out through 'npm bin' command
npm_bin_path = get_ipython().getoutput('npm bin')[0]
print('npm bin path: ', npm_bin_path)

# Run `azurite-blob -s` as a background process. 
# IPython doesn't recognize `&` in inline bash cells.
get_ipython().system_raw(npm_bin_path + '/' + 'azurite-blob -s &')

npm bin path:  /content/node_modules/.bin


### TensorFlow を使用した Azure Storage のファイルの読み取りと書き込み

以下は、TensorFlow API を使用して、Azure Storage のファイルの読み取りと書き込みを行う例です。

`tensorflow-io` は自動的に `azfs` の使用を登録するため、`tensorflow-io` パッケージがインポートされると、ほかのファイルシステム（POSIX または GCS）と同じように動作します。

Azure Storage キーは、`TF_AZURE_STORAGE_KEY` 環境変数で指定します。これを行わない場合、`TF_AZURE_USE_DEV_STORAGE` は `True` に設定され、代わりに Azurite エミュレータが使用されてしまいます。


In [ ]:
import os
import tensorflow as tf
import tensorflow_io as tfio

# Switch to False to use Azure Storage instead:
use_emulator = True

if use_emulator:
  os.environ['TF_AZURE_USE_DEV_STORAGE'] = '1'
  account_name = 'devstoreaccount1'
else:
  # Replace <key> with Azure Storage Key, and <account> with Azure Storage Account
  os.environ['TF_AZURE_STORAGE_KEY'] = '<key>'
  account_name = '<account>'

  # Alternatively, you can use a shared access signature (SAS) to authenticate with the Azure Storage Account
  os.environ['TF_AZURE_STORAGE_SAS'] = '<your sas>'
  account_name = '<account>'

In [6]:
pathname = 'az://{}/aztest'.format(account_name)
tf.io.gfile.mkdir(pathname)

filename = pathname + '/hello.txt'
with tf.io.gfile.GFile(filename, mode='w') as w:
  w.write("Hello, world!")

with tf.io.gfile.GFile(filename, mode='r') as r:
  print(r.read())

Hello, world!


## 構成

TensorFlow での Azure Blob Storage の構成は、常に環境変数を通じて行われます。次に利用できるすべての構成を示します。

- `TF_AZURE_USE_DEV_STORAGE`: 'az://devstoreaccount1/container/file.txt' などの接続にローカル開発ストレージエミュレータを使用する場合は、1 に設定します。これはほかのすべての設定よりも優先されるため、ほかの接続を使用するには `unset` にしてください。
- `TF_AZURE_STORAGE_KEY`: 使用中の Storage アカウントのアカウントキー
- `TF_AZURE_STORAGE_USE_HTTP`: https 転送を使用しない場合は、任意の値に設定します。デフォルトの hrrps を使用する場合は `unset` にしてください。
- `TF_AZURE_STORAGE_BLOB_ENDPOINT`: BLOB ストレージのエンドポイントに設定します。デフォルトは `.core.windows.net` です。
